# DNBLab Jupyter Notebook Tutorial

## SRU - Schnittstellenabfrage, Datenauslieferung und Ergebnisanzeige

Dieses DNBLab-Tutorial beschreibt eine Beispielabfrage über die SRU-Schnittstelle mit Python. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Das Tutorial umfasst die exemplarische Anfrage und Ausgabe der Daten in MARC21-xml zur weiteren Verarbeitung. 

Die Daten können über die SRU-Schnittstelle als XML-Antwort in folgenden Formaten ausgeliefert werden: 
* MARC21-xml http://www.loc.gov/standards/marcxml/schema/MARC21slim.xsd
* DNB Casual (oai_dc)	http://www.openarchives.org/OAI/2.0/oai_dc.xsd
* RDF (RDFxml) http://www.w3.org/2000/07/rdf.xsd

## Einrichten der Arbeitsumgebung <a class="anchor" id="Teil1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Bibliotheken importiert werden. Für Anfragen über die SRU-Schnittstelle wird BeautifulSoup https://www.crummy.com/software/BeautifulSoup/ und zur Verarbeitung der XML-Daten etree https://docs.python.org/3/library/xml.etree.elementtree.html verwendet. Mit Pandas https://pandas.pydata.org/ können Elemente aus dem MARC21-Format ausgelesen werden.

In [5]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

## SRU-Abfrage mit Ausgabe in MARC21-xml<a class="anchor" id="Teil2"></a>

Die Funktion dnb_sru nimmt den Paramter "query" der SRU-Abfrage entgegen und liefert alle Ergebnisse als eine Liste von Records aus. Bei mehr als 100 Records werden weitere Datensätze mit "&startRecord=101" abgerufen (mögliche Werte 1 bis 99.000). Weitere Informationen und Funktionen der SRU- Schnittstelle werden unter https://www.dnb.de/sru beschrieben.

In [6]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

## Durchsuchen eines MARC-Feldes<a class="anchor" id="Teil3"></a>

Die Funktion parse_records nimmt als Parameter jeweils ein Record entgegen und sucht über xpath die gewünschte Informationen heraus und liefert diese als Dictionary zurück. Die Schlüssel-Werte-Paare können beliebig agepasst und erweitert werden. In diesem Fall werden Elemente für IDN und Titel geliefert.

In [7]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        titel = titel[0].text
        #umfang = unicodedata.normalize("NFC", umfang)
    except:
        titel = "unkown"
        
    meta_dict = {"idn":idn,
                 "titel":titel}
    
    return meta_dict

Über die verschiedenen Indices https://services.dnb.de/sru/dnb?operation=explain&version=1.1 kann die SRU-Abfrage mittels CQL gezielt z.B. über das Titelstichwort "Klimawandel" in Kombination mit dem Standort "online frei verfügbar" eingeschränkt werden. Auf diese Art kann durch Anpassen des Codes nach verschiedenen Begriffen in beliebigen MARC-Feldern gesucht werden. 

In [8]:
records = dnb_sru('tit=Klimawandel and location=onlinefree')
print(len(records), 'Ergebnisse')

468 Ergebnisse


## Beispielanzeige zur weiteren Bearbeitung <a class="anchor" id="Teil4"></a>

Mit der "Python Data Analysis Library" Pandas für Python werden die Ergebnisse (Dictionary-Elemente) als Dataframe ausgegeben. Die ersten 5 und letzten 5 Zeilen des Dataframes können mit dem Befehl "df = pd.DataFrame() angezeigt werden. 

In [9]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

,idn,titel
0,1230754067,Das Klima macht uns krank!
1,1230754121,Fällt uns das Dach der Welt bald auf den Kopf?
2,1230754253,Folgen des Klimawandel in Mitteleuropa
3,1228535671,How does climate change affect human behavior?...
4,123123976X,Identifizierung von für Deutschland relevanten...
...,...,...
463,1127681362,Namibia: Biodiversitätsmanagement und Klimawandel
464,1228308977,Naturschutz und Klimawandel im Leipziger Auwald
465,1217229078,Neue Herausforderungen für die Anästhesie durc...
466,1142648214,Peru Leben mit dem Klimawandel


Mit der folgenden Funktion df.to_csv() werden die Ergebnisse als SRU_Titel.csv in das Jupyter-Verzeichnins der Einstiegsseite abgelegt und können dort heruntergeladen werden. 

In [10]:
df.to_csv("SRU_Titel.csv", index=False)